# Baseline model
Taxi demand prediction

In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0


In [6]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(
    f'X_train shape: {X_train.shape}\n'
    f'X_test shape: {X_test.shape}\n'
    f'y_train shape: {y_train.shape}\n'
    f'y_test shape: {y_test.shape}\n'
)

X_train shape: (32226, 674)
X_test shape: (56068, 674)
y_train shape: (32226,)
y_test shape: (56068,)


## Create a baseline models

In [3]:
import numpy as np

class BaselineModelPreviousHour:
    """Baseline model that predicts the previous one hour's value."""
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test['rides_previous_1_hour']

In [7]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
predictions

0         0.0
1         5.0
2        13.0
3        12.0
4        14.0
         ... 
56063     0.0
56064     0.0
56065     0.0
56066     0.0
56067     0.0
Name: rides_previous_1_hour, Length: 56068, dtype: float32

In [9]:
from sklearn.metrics import mean_absolute_error
# hide warnings
import warnings
warnings.filterwarnings('ignore')

test_mea = mean_absolute_error(y_test, predictions)
print(f'Test MAE: {test_mea:.4f}')

Test MAE: 6.1252


## Baseline model that predicts the demand observed previous week

In [10]:
import pandas as pd
import numpy as np

class BaselinePreviousWeek:
    """Baseline model that predicts the demand observed previous week."""
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f'rides_previous_{7*24}_hour']

In [11]:
model = BaselinePreviousWeek()
predictions = model.predict(X_test)

In [12]:
test_mea = mean_absolute_error(y_test, predictions)
print(f'Test MAE: {test_mea:.4f}')

Test MAE: 3.7232


### Baseline model 
That predicts the demand observed at
 - t - 7 days
 - t - 14 days
 - t - 21 days
 -  t - 28 days

In [13]:
class BaselineModelLast4Weeks:
    """Baseline model that predicts the demand observed at t - 7 days, t - 14 days, t - 21 days, and t - 28 days."""
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass
    
    def predict(self, X_test: pd.DataFrame) -> np.array:
        return 0.25*(
            X_test[f'rides_previous_{7*24}_hour'] +
            X_test[f'rides_previous_{7*24*2}_hour'] +
            X_test[f'rides_previous_{7*24*3}_hour'] +
            X_test[f'rides_previous_{7*24*4}_hour']
        )

In [14]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [15]:
test_mea = mean_absolute_error(y_test, predictions)
print(f'Test MAE: {test_mea:.4f}')

Test MAE: 3.2329
